# Byte #3: Function Calling with OpenAI (Agents in Action) 🔧

**⏱️ Time to Complete: 10-15 minutes**

Welcome back to hero HQ! By now you can talk to models, reason through async calls, and keep conversations alive. In this byte you will teach an assistant to reach beyond the model and use trusted tools—perfect for building production-ready copilots that can look up data, run calculations, or trigger workflows on demand.

## Why Function Calling?

Models don’t automatically know your private knowledge base, inventory numbers, or the latest sensor readings. Function calling lets you expose safe, well-defined functions so an assistant can fetch facts, perform actions, or orchestrate other systems—all while you stay in control of what can be executed.

### Core Concepts

- **Tool** → A function you register with the model (JSON schema describes inputs).
- **Tool call** → The model requests a tool by name with arguments when it needs external help.
- **Tool result** → Your application executes the function, then replies with the output so the model can finish its answer.
- **Guardrails** → You decide which tools exist and validate every call before running it.

### How Tool Calling Works (Quick View)

1. You describe every tool with JSON Schema so the assistant knows the contract.
2. The model decides whether to call a tool and returns the requested name plus arguments.
3. Your code validates the request, executes the function, and packages the result.
4. You send that tool output back through the `input` list so the model can finish its answer.

_The assistant never runs code on its own—you stay in the loop for every call._

## Setup Checklist

1. Activate your project virtual environment.
2. Install dependencies once with `uv pip install -e .` so the OpenAI SDK and dotenv helper are ready.
3. Confirm your `.env` file has a valid `OPENAI_API_KEY` (and restart the kernel after edits).

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import json

# Load environment variables so the OpenAI SDK can find OPENAI_API_KEY
load_dotenv()
client = OpenAI()
print("✅ OpenAI client ready for tool calling")

## Step 1: Why Use Tools for Math?

Large language models are fluent with words but can drift when arithmetic gets tricky. Asking for the cube root of 343 and the square root of 576 sounds simple, yet minor sampling randomness or a rushed internal calculation can still produce mistakes. Instead of trusting the guess, we can hand the computation to deterministic Python helpers via tool calling and let the model focus on the conversation.

In [ ]:
import math

def calculate_root(operation: str, value: float) -> dict:
    """Return precise square or cube roots using Python math helpers."""
    if operation == "square_root":
        result = math.sqrt(value)
    elif operation == "cube_root":
        result = math.copysign(abs(value) ** (1 / 3), value)
    else:
        raise ValueError(f"Unsupported operation: {operation}")
    return {
        "operation": operation,
        "value": value,
        "result": result,
    }

AVAILABLE_TOOLS = {"calculate_root": calculate_root}
print("🛠️ Local tools registered: " + ", ".join(AVAILABLE_TOOLS.keys()))

## Step 2: Describe the Math Tool

The assistant needs a precise contract to know how to call `calculate_root`. We'll publish a JSON Schema that constrains the `operation` to either `square_root` or `cube_root` and requires a numeric `value`.

In [ ]:
tools = [
    {
        "type": "function",
        "name": "calculate_root",
        "description": "Compute the square root or cube root of a number using Python math.",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["square_root", "cube_root"],
                    "description": "Choose whether to calculate a square root or cube root.",
                },
                "value": {
                    "type": "number",
                    "description": "Positive or negative number to process.",
                },
            },
            "required": ["operation", "value"],
        },
    },
 ]
print(json.dumps(tools, indent=2))

## Step 3: Let the Assistant Delegate the Roots

We'll ask for two values in one sentence: the square root of 576 and the cube root of 343. The model will parse the request, decide to call `calculate_root` for each portion, and we will execute the calculations so the final reply is exact.

In [ ]:
question = "Please share the square root of 576 and the cube root of 343."

# Create a running input list we will add to over time
input_list = [
    {
        "role": "user",
        "content": [
            {"type": "input_text", "text": question}
        ],
    }
 ]

# First round-trip: ask the model for help and let it decide if tools are needed
response = client.responses.create(
    model="gpt-5-nano",
    tools=tools,
    input=input_list,
 )

def _serialize_output_item(item):
    """Convert SDK output objects into dictionaries we can send back to the API."""
    payload = json.loads(item.model_dump_json())
    payload.pop("status", None)
    return payload

# Append everything the model just produced (tool calls, thoughts, etc.) back into the transcript
input_list += [_serialize_output_item(item) for item in response.output]

# Execute each tool request locally and push the results to the transcript
for item in response.output:
    if item.type == "function_call" and item.name == "calculate_root":
        args = item.arguments
        if isinstance(args, str):
            args = json.loads(args or "{}")
        elif args is None:
            args = {}
        tool_result = calculate_root(**args)
        input_list.append({
            "type": "function_call_output",
            "call_id": item.call_id,
            "output": json.dumps(tool_result),
        })

print("Final input transcript:")
print(json.dumps(input_list, indent=2))


In [ ]:

# Second round-trip: resend the enriched transcript so the model can compose the final answer
response = client.responses.create(
    model="gpt-5-nano",
    instructions="Summarize the tool outputs with the exact numeric answers.",
    tools=tools,
    input=input_list,
 )

print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + (response.output_text or ""))

### Flow Recap

1. **Send** the user request along with the `calculate_root` schema.
2. **Inspect** the assistant output—each `function_call` includes the operation (`square_root` or `cube_root`) plus the numeric value it extracted from the question.
3. **Respond** by running the Python helper and pushing a `function_call_output` back into the transcript.
4. **Read** the assistant’s final narrative, which now quotes the precise results computed in Python.

### Step-by-Step Decision Trace

1. **Tool awareness**: Because the schema lists only `calculate_root`, the model knows that any precise math should go through that function.
2. **Argument extraction**: While generating its first response, the model spots “square root of 576” and “cube root of 343,” mapping them to two calls with `operation` and `value` arguments.
3. **Your execution**: Python evaluates both requests—`math.sqrt(576)` and the signed cube root of 343—returning exact floats.
4. **Final message**: With those outputs stitched into the transcript, the model crafts a natural-language answer that cites the tool data instead of guessing.

## Step 4: Wrap It in a Helper

Encapsulate the pattern so services can reuse it. The helper below runs the complete loop and returns both the structured tool outputs and the assistant narration, guaranteeing every root comes from Python instead of a language-model guess.

In [ ]:
def _parse_arguments(raw_args):
    if isinstance(raw_args, str):
        raw_args = raw_args or "{}"
        return json.loads(raw_args)
    return raw_args or {}

def _serialize_output_item(item):
    payload = json.loads(item.model_dump_json())
    payload.pop("status", None)
    return payload

def run_assistant(prompt: str, model: str = "gpt-5-nano") -> dict:
    """Run the Responses API tool-calling loop and return structured data."""
    transcript = []
    input_list = [
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt}
            ],
        }
    ]

    initial = client.responses.create(
        model=model,
        tools=tools,
        input=input_list,
    )
    transcript.append(initial)
    input_list += [_serialize_output_item(item) for item in initial.output]

    call_outputs = []
    for item in initial.output:
        if item.type == "function_call":
            tool_fn = AVAILABLE_TOOLS.get(item.name)
            args = _parse_arguments(item.arguments)
            if not tool_fn:
                result_payload = {"success": False, "error": f"Unknown tool: {item.name}"}
            else:
                try:
                    result_payload = tool_fn(**args)
                except TypeError as exc:
                    result_payload = {"success": False, "error": str(exc)}
            serialized_result = json.dumps(result_payload)
            call_outputs.append({
                "call_id": item.call_id,
                "output": serialized_result,
            })
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": serialized_result,
            })

    final = client.responses.create(
        model=model,
        tools=tools,
        instructions="Summarize the tool findings clearly for the user.",
        input=input_list,
    )
    transcript.append(final)

    return {
        "prompt": prompt,
        "input_list": input_list,
        "tool_outputs": call_outputs,
        "assistant_text": final.output_text,
        "responses": transcript,
    }

demo = run_assistant("What are the square root of 1024 and the cube root of 64?")
print(demo["assistant_text"])

## Optional: Async Version

Large agents often run in event loops. The async helper mirrors the same flow but awaits each network hop so you can orchestrate multiple assistants concurrently.

In [ ]:
from openai import AsyncOpenAI

async_client = AsyncOpenAI()

async def run_assistant_async(prompt: str, model: str = "gpt-5-nano") -> dict:
    """Async variant that mirrors the cumulative input workflow."""
    transcript = []
    input_list = [
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt}
            ],
        }
    ]

    initial = await async_client.responses.create(
        model=model,
        tools=tools,
        input=input_list,
    )
    transcript.append(initial)
    input_list += [_serialize_output_item(item) for item in initial.output]

    call_outputs = []
    for item in initial.output:
        if item.type == "function_call":
            tool_fn = AVAILABLE_TOOLS.get(item.name)
            if not tool_fn:
                continue
            args = _parse_arguments(item.arguments)
            try:
                result_payload = tool_fn(**args)
            except TypeError as exc:
                result_payload = {"success": False, "error": str(exc)}
            json_payload = json.dumps(result_payload)
            call_outputs.append({
                "call_id": item.call_id,
                "output": json_payload,
            })
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json_payload,
            })

    final = await async_client.responses.create(
        model=model,
        tools=tools,
        instructions="Summarize the tool findings clearly for the user.",
        input=input_list,
    )
    transcript.append(final)

    return {
        "prompt": prompt,
        "input_list": input_list,
        "tool_outputs": call_outputs,
        "assistant_text": final.output_text,
        "responses": transcript,
    }

async_demo = await run_assistant_async("Calculate the square root of 361 and the cube root of -512.")
print(async_demo["assistant_text"])

## When Custom Tools Aren't Needed

Many workflows already ship in the Responses API. After completing the math walkthrough, scroll to the end of this notebook for the built-in `web_search` demo. It highlights how you can mix and match: rely on turnkey tools like `web_search`, `code_interpreter`, or `file_search` for standard capabilities, and reserve custom functions such as `calculate_root` for domain-specific logic.

## Built-In Tools in Action

You do not need to craft a new function for every capability. The Responses API exposes several built-in tools you can toggle on when deploying assistants. The example below reuses `web_search` to surface the latest quantum-computing headline—no custom Python required aside from recording the result. (Feature availability varies by region, so run this cell to confirm access in your account.)


In [ ]:
web_search_demo = client.responses.create(
    model="gpt-4.1-mini",
    tools=[{"type": "web_search"}],
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "Find one headline-worthy quantum computing breakthrough from this month and summarize it.",
                }
            ],
        }
    ],
)
print(web_search_demo.model_dump_json(indent=2))
print("\nSummary:")
print(web_search_demo.output_text or "(No text output)")

## ✅ Key Takeaways

✓ Tool calling bridges model creativity with your live systems.

✓ Built-in tools like `web_search`, `code_interpreter`, and `file_search` give you zero-setup superpowers—mix them with your custom functions for richer assistants.

✓ Always describe tools precisely with JSON Schema so the assistant knows when to use them.

✓ You must execute tool logic yourself—validate inputs, handle errors, and return structured output.

✓ Wrap the request → tool → response loop in helpers to keep production agents tidy.

—

**Great work!** You now have the blueprint for assistants that can reason, fetch, and act with confidence.